# Volume action testing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from armscan_env import config
from armscan_env.clustering import TissueClusters
from armscan_env.envs.rewards import anatomy_based_rwd
from armscan_env.envs.state_action import ManipulatorAction
from armscan_env.util.visualizations import show_clusters
from armscan_env.volumes.loading import (
    load_sitk_volumes,
    normalize_sitk_volumes_to_highest_spacing,
)
from armscan_env.volumes.volumes import TransformedVolume

config = config.get_config()

Let's load all the volumes in their original shape and in the normalized shape:

In [ ]:
volumes = load_sitk_volumes(normalize=False)
cropped_volumes = load_sitk_volumes(normalize=True, cropped=True)
plt.style.use("default")
normalized_volumes = normalize_sitk_volumes_to_highest_spacing(volumes)
print(volumes[0].GetSize())
print(normalized_volumes[0].GetSize())

Now you can set as `volume` any of the volumes(normalized or not):

In [ ]:
volume_1 = sitk.ReadImage(config.get_single_labelmap_path(37))
volume_img_1 = sitk.GetArrayFromImage(volume_1)
size = np.array(volume_1.GetSize()) * np.array(volume_1.GetSpacing())
print(f"{size=} mm")
transversal_extent = (0, size[0], 0, size[2])
longitudinal_extent = (0, size[1], 0, size[2])
frontal_extent = (0, size[0], size[1], 0)

In [ ]:
cropped_volume_img_1 = sitk.GetArrayFromImage(cropped_volumes[0])
cropped_volume_img_2 = sitk.GetArrayFromImage(cropped_volumes[1])
cropped_volume_img_3 = sitk.GetArrayFromImage(cropped_volumes[2])
cropped_volume_img_4 = sitk.GetArrayFromImage(cropped_volumes[3])

fig, axs = plt.subplots(2, 2, figsize=(12, 5))
axs[0, 0].imshow(cropped_volume_img_1[41, :, :])
axs[0, 1].imshow(cropped_volume_img_2[51, :, :])
axs[1, 0].imshow(cropped_volume_img_3[40, :, :])
axs[1, 1].imshow(cropped_volume_img_4[35, :, :])
# turn ogg axis for all subplots
for ax in axs.flat:
    ax.axis("off")
plt.show()

In [ ]:
optimal_action = ManipulatorAction(rotation=(0, 0), translation=(0, 140))

plt.imshow(volume_img_1[50, :, :], extent=frontal_extent)
o = volume_1.GetOrigin()
x_dash = np.arange(size[0])
b = optimal_action.translation[1]
y_dash = x_dash * np.tan(np.deg2rad(optimal_action.rotation[0])) + b
plt.plot(x_dash, y_dash, linestyle="--", color="red")

plt.show()

In [ ]:
sliced_volume = volume_1.get_volume_slice(
    action=volume_1.optimal_action,
    slice_shape=(volume_1.GetSize()[0], volume_1.GetSize()[2]),
)
sliced_img = sitk.GetArrayFromImage(sliced_volume)
print(f"Slice value range: {np.min(sliced_img)} - {np.max(sliced_img)}")

extent_xz = (0, size[0], size[2], 0)
cluster = TissueClusters.from_labelmap_slice(sliced_img.T)
show_clusters(cluster, sliced_img.T)
reward = anatomy_based_rwd(cluster)
print(f"Reward: {reward}")
plt.axis("off")
plt.show()

In [ ]:
volume_transformation = ManipulatorAction(
    rotation=(-7.213170270886784, 0.0),
    translation=(-7.31243280019082, 9.172539411055304),
)
transformed_volume = TransformedVolume(volume_1, volume_transformation)
transformed_action = transformed_volume.optimal_action
print(f"{volume_1.optimal_action=}\n{transformed_volume.optimal_action=}\n")

In [ ]:
transformed_img = sitk.GetArrayFromImage(transformed_volume)

plt.imshow(transformed_img[40, :, :], extent=transversal_extent)

ot = transformed_volume.GetOrigin()
x_dash = np.arange(size[0])
b = transformed_action.translation[1]
y_dash = x_dash * np.tan(np.deg2rad(transformed_action.rotation[0])) + b
plt.plot(x_dash, y_dash, linestyle="--", color="red")

plt.show()

In [ ]:
sliced_transformed_volume = transformed_volume.get_volume_slice(
    action=transformed_action,
    slice_shape=(volume_1.GetSize()[0], volume_1.GetSize()[2]),
)
sliced_transformed_img = sitk.GetArrayFromImage(sliced_transformed_volume)
print(f"Slice value range: {np.min(sliced_transformed_img)} - {np.max(sliced_transformed_img)}")

cluster = TissueClusters.from_labelmap_slice(sliced_transformed_img.T)
show_clusters(cluster, sliced_transformed_img.T)
reward = anatomy_based_rwd(cluster)
print(f"Reward: {reward}")
plt.show()